In [1]:
#Loading libraries
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

#### Implement function ridge_regression_data that computes the ridge regression data matrix

In [2]:
def ridge_regression_data(data_inputs, degree=1):
    if (degree>1 and data_inputs.ndim>1):
        raise NotImplementedError()
    if (data_inputs.ndim==1):
        d_mat=np.ones((len(data_inputs),1))
        for i in range(degree):
            d_mat=np.c_[d_mat,np.power(data_inputs,i+1)]
    else:
        first_c=np.ones((len(data_inputs),1))
        d_mat=np.c_[first_c,data_inputs]
    return d_mat

In [3]:
test_inputs = np.array([[1, 2], [2, 3], [3, 4], [4, 5]])
ridge_regression_data(test_inputs)

array([[1., 1., 2.],
       [1., 2., 3.],
       [1., 3., 4.],
       [1., 4., 5.]])

Function **ridge_regression** takes three arguments *data_matrix*, *data_outputs* and *regularisation*, which computes and returns the solution $\hat{\mathbf{W}}$ of the normal equation
$
\left(\Phi^{\top}\left(\mathbf{X}\right)\Phi\left(\mathbf{X}\right) +\alpha I \right)\hat{\mathbf{W}} = \Phi^{\top}\left(\mathbf{X}\right)\mathbf{Y}
$

In [4]:
def ridge_regression(data_matrix, data_outputs, regularisation=0):
    P=data_matrix
    a=regularisation
    Y=data_outputs
    Id=np.identity(len(data_matrix.T))
    return np.linalg.solve(P.T @ P + a * Id, P.T @ Y)

In [5]:
test_inputs = np.array([[0.98], [1.02]])
test_outputs = np.array([[-0.1], [0.3]])
test_data_matrix = ridge_regression_data(test_inputs)
regularisation = 0.5
ridge_regression(test_data_matrix, test_outputs, regularisation)

array([[0.03737123],
       [0.05328597]])

**prediction_function**  evaluates the predicted regression function at given points $\mathbf{X} = 
\left\{x^{(1)}, x^{(2)}, \ldots, x^{(s)}\right\}$ for given coefficients $\mathbf{W} = \left(w^{(0)}, w^{(1)},\ldots,w^{(d)}\right)$

In [6]:
def prediction_function(data_matrix, weights):
    return data_matrix@weights

In [7]:
test_inputs = np.array([[1, 2, 3]])
test_weights = np.array([[-1, 1], [2, 2], [-3, 3], [4, 4]])
test_data_matrix = ridge_regression_data(test_inputs)
prediction_function(test_data_matrix, test_weights)

array([[ 7., 21.]])

Function **prediction_error** evaluates a mean-squared error over the set of data inputs and outputs

In [8]:
def prediction_error(data_matrix, data_outputs, weights):
    s=len(data_matrix)
    P=data_matrix
    W=weights
    Y=data_outputs
    return 1 / (2 * s) * (np.linalg.norm(P @ W - Y))**2

In [9]:
test_inputs = np.array([[1, -1], [2, 2]])
test_data_outputs = np.array([[-1, 2], [1, 3]])
test_data_matrix = ridge_regression_data(test_inputs)
test_weights = np.array([[0, 0], [1, 2], [3, 4]])
prediction_error(test_data_matrix, test_data_outputs, test_weights)

36.75

### K-fold cross validation

Function **KFold_split** takes two arguments *data_size* and *K* and outputs a random split of integer indexes $\left[0,1,\ldots,data\_size\right)$ into $K$ almost equal chunks.

In [10]:
def KFold_split(data_size, K):
    np.random.seed(212)
    ind = np.random.permutation(data_size)
    m, r = divmod(data_size, K)
    ind_split = [ind[i * m + min(i,r):(i + 1) * m + min(i + 1, r)]for i in range(K)]
    return ind_split

The function **KFold_cross_validation** should return a matrix/vector of coefficients/weights and a validation error that are both evaluated as averages of optimal weights and validation error of every iteration step.

In [11]:
def KFold_cross_validation(data_matrix, data_outputs, K, model_evaluation,
                           error_evaluation):
    data_size = len(data_matrix)
    ind_split = KFold_split(data_size, K)
    for i in range(K):
        indexes = np.concatenate([ind_split[j] for j in range(K) if (j != i)])
        weights = model_evaluation(data_matrix[indexes], data_outputs[indexes])
        error = error_evaluation(data_matrix[ind_split[i]],
                             data_outputs[ind_split[i]], weights)
        if (i == 0):
            optimal_weights = weights / K
            validation_error = error / K
        else:
            optimal_weights += weights / K
            validation_error += error / K
    return optimal_weights, validation_error

### Data standardisation

In [12]:
def standardise(data_matrix):
    row_means = np.mean(data_matrix, axis=0)
    std_matrix = data_matrix - row_means
    row_stds = np.std(std_matrix, axis=0)
    return (std_matrix / row_stds), row_means, row_stds

In [14]:
def de_standardise(std_matrix, row_means, row_stds):
    matrix = np.copy(std_matrix * row_stds)
    return matrix + row_means

### Apply all the above tools for training a model predicting housing prices in Boston

In [15]:
#Loading the data

housing_dataset_path = "house_prices.csv"
housing_data = np.genfromtxt(housing_dataset_path,
                             delimiter=",",
                             skip_header=1,
                             usecols=[0, 1, 2, 3, 4, 5, 6, 7])
housing_data_input = housing_data[:, 0:7]
housing_data_output = housing_data[:, 7]

In [17]:
std_input, input_mean, input_SD = standardise(housing_data_input)
std_output, output_mean, output_SD = standardise( housing_data_output)

In [18]:
#grid_search performs a search for a minimum value of a given function on a given grid points
def grid_search(objective, grid):
    values = np.array([])
    for point in grid:
        values = np.append(values, objective(point))
    return grid[np.argmin(values)]

In [20]:
K = 5
alpha_grid = np.append(np.array([i * 0.05 for i in range(20)]),
                       np.array([i for i in range(1, 20)]))

validation_error = lambda alpha: KFold_cross_validation(s_input, s_output, K,\
                                                        lambda data_matrix, data_outputs: ridge_regression(data_matrix, data_outputs, regularisation =alpha),\
                                                       lambda data_matrix, data_outputs, weights: prediction_error(data_matrix, data_outputs, weights))[1]

optimal_alpha = grid_search(validation_error, alpha_grid)

optimal_weights = KFold_cross_validation(s_input, s_output, K,\
lambda data_matrix, data_outputs: ridge_regression(data_matrix, data_outputs, regularisation = optimal_alpha),\
lambda data_matrix, data_outputs, weights: prediction_error(data_matrix, data_outputs, weights))[0]

print("An optimal value of regularisation parameter is {}.\nFor this value of regularisation parameter one gets optimal weights of the form \n{}"
    .format(optimal_alpha, optimal_weights))

An optimal value of regularisation parameter is 19.0.
For this value of regularisation parameter one gets optimal weights of the form 
[-0.00682014  0.06899114  0.44764439  0.03139397  0.15058687  0.26429561
  0.15906673]
